In [5]:
# ToDo:

# Have working on Colab ON HOLD

# General usability. PROGRESS

# General form layout imprvements. PROGRESS

# Make "Here is the text: " hardcoded at the end of the users labelling text/

# Sort the placeholder ordering to reflect the order in the template

# General code quality
# - probably some repeated code between forms that could be refactored
# - much of the form code stitched together from LLM generated code for fast prototyping
# --- Is it all necessary?
# --- Can it be improved?
# --- Does it make sense?
# TODO Aayush to Matthew: It makes sense to do this, though I'd say after we get the usability stuff done? (forms, usability improvements, directory of labels etc.). Since this is mainly a backend improvement, and the notebook users won't care.

# Track cost of label generation

# Prompt novelty
# --- random seed but record seeds
# --- length
# --- Sample from top 5?
# --- higher shot in k-shot
# --- bigger model - more expensive

# Pick up from where we left off of API call fails
# Do you loose everything if you stop the API call?
# TODO: Aayush to Matthew: How important of a problem is this to solve?
# Matthew to Aayush: Not very important.

# Notebook explanation

The purpose of this notebook is to use a prompt given to some LLM ("LLM A") to generate a dataset of different prompts to be fed to some other LLM ("LLM B" - potentially the same as "LLM A") for which you have access to the weights. This dataset of prompts will be used to create a "representation vector" of a property or concept for LLM B in order to "steer" LLM B to have more of that property or concept in its outputs. This Notebook does not cover the creation of representations or steering, only the the creation of a dataset of prompts.

The kinds of prompts you want to generate should be about the property or concept you want to steer the LLM towards, not necessary literally mentioning it - e.g. prompts about politeness do not necessarily need to have the word polite in them. Part of the point of dataset creation is to explore which kinds of generated prompts yield good representations.

The Notebook works as follows:

- The Setup section loads Python libraries needed to run the code. You do not need to change anything here.
- The Inputs section is where you define the prompt you will use to generate your dataset of prompts. Instructions on how to do this are given. This is the only section of the notebook where you will need to change anything.
- The Review section then generates a small example dataset of prompts and shows them to you. If you like them, continue on to the end of the Notebook.
- If you do not like them, please go back to the Inputs section to refine your prompt to generate your dataset of prompts.
- The Dataset Generation section completes the dataset generation.
- The View Dataset section loads your generated dataset for inspection.
- Your dataset will be stored in /data/inputs/name_of_your_dataset/dataset if you want to use it later.

The datasets will be generated in CSV format and should have the following form, where the first line is the column headings and subsequent lines are for example prompts. The columns ethical_area and ethical_valence are two different labels (classifications) of the prompt. The Notebook will help you generate these labels. This example is for "politeness" prompts:

```
prompt, ethical_area, ethical_valence
"Would you be so kind as to pass the water please.", "polite", 1
"Give me the water now.", "impolite", 0
```

If this is too restrictive, you will be able to create other columns and the notebook will ask you about that too.

Note that where we have text, such as for the prompt or ethical_area, we want it enclosed in quote marks. Lone numbers do not need this.

If there is something unusual you want to do that the current notebook does not permist, please ask, or feel free to try adding code for it. 

# Setup (just run)

In [1]:
import csv
import json
import math
import os
import re
import time
from IPython import get_ipython
from IPython.display import display, HTML, clear_output
from ipywidgets import (
    Button,
    Checkbox,
    Dropdown,
    FloatText,
    HBox,
    IntText,
    Label,
    Layout,
    Output,
    SelectMultiple,
    Text,
    Textarea,
    VBox,
    interactive,
    interact,
    interact_manual,
    fixed,
    widgets,
)
from jinja2 import Environment, FileSystemLoader, meta
from hydra import initialize
from hydra.core.global_hydra import GlobalHydra
from hydra.experimental import compose
from omegaconf import DictConfig, OmegaConf
from openai import OpenAI
import pandas as pd
import yaml

from dataset_creator_helper import NotebookHelper, DatasetGenerator, DataGenerator
from dataset_creator_ui_components import UIHelper


# Basic Settings

The form below two cells below should allow you to enter your open AI API key on the form, or it will find it in your environment variables if you set it.

However, on some machines the form did not allow users to enter their open AI API key. If you did not set your OpenAI key in your environment variables, and you are having trouble entering you open AI API key on the form, enter it in the cell below first and the for, should load it automatically.

Please do not commit and push code with live API keys.

In [2]:
# You only need to enter your API key if you are having issues with the form.
# Uncomment the line (remove the # symbol) below and replace 'your_openai_api_key' with your actual key.

# os.environ['OPENAI_API_KEY'] = 'your_openai_api_key'

In [3]:
nh = NotebookHelper(api_key="your_openai_api_key_here")
ui = UIHelper(nh)

Using api_key sk-... Update below if incorrect.


In [4]:
ui.display_ui()

Dropdown(description='OpenAI Model:', index=1, options=('gpt-4-0125-preview', 'gpt-4', 'gpt-3.5-turbo'), style…

FloatSlider(value=1.0, continuous_update=False, description='Temperature:', max=2.0, readout_format='.1f', sty…

Text(value='new_template')

IntText(value=100, description='Total Number of Examples:', style=DescriptionStyle(description_width='initial'…

IntText(value=100, description='Examples per Request:', style=DescriptionStyle(description_width='initial'))

API Key: sk-...
OpenAI Model: gpt-4
Temperature: 1.0
Filename: testtest.csv
Total Number of Examples: 100
Examples per Request: 100
----------
API key "sk-..." saved to environment variables.
API Key: sk-...
OpenAI Model: gpt-4
Temperature: 1.0
Filename: new_template.csv
Total Number of Examples: 100
Examples per Request: 100
----------


## Hints

**OpenAI Api Key**
- Get an API key [here](https://auth0.openai.com/u/signup/identifier?state=hKFo2SA1REJ3dGFZVTllNHFvYUFkY2RrWEJpUUVMUWxvel91VqFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIFg1Z2NKOU9hUk4yYUFmWGxyTHlscmtNTmMxbDF5dWZTo2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q).
- You will need to put some token amount of money on to use gpt-4-0125-preview. It's something like $1. Might be $10.

**OpenAI Model**
- See [here](https://platform.openai.com/docs/models/) for the list of models: 

**Temperature**
- Lower values for temperature result in more consistent outputs (e.g. 0.2), while higher values generate more diverse and creative results (e.g. 1.0)
- If your generated sentences are too similar/boring, try increasing the number. Go the other way if too wacky.
- More details [here](https://platform.openai.com/docs/guides/text-generation/how-should-i-set-the-temperature-parameter)

**Filename**
- Saves the dataset to a file with this name.
- Try to give it a specific name e.g. "honesty_v2.csv" or "honesty_pairs_v2.csv", something that will help you remember what it is especially if you are experimenting with variations.
- Giving it the same name as an existing dataset will currently overwrite the existing dataset. TODO: Resolve this.

**Total Number of Examples**
- This is the total number of prompts you want to generate in your dataset of prompts.
- If you want to generate x prompts in total, put x here.
- What is the max number of examples one should put here?

**Number of Examples per request**
- TODO: @Matthew should we get rid of this variable since Rasmus wants everything in one batch, or is there still usefulness to it?
- This is the number of prompt examples that are generated per request to the model.
- The reason we have this is because the LLM has a limit on the number of words it can process in one request. If you put a number that is too high here, you will not get the number of prompts you want.




TODO: Kept this text just in case for now.

Enter below the number of examples per request. This is the number of prompt examples you want to generate in one request to the LLM.

- This is different from the total_examples variable you entered above.
- The reason we have this is because the LLM has a limit on the number of tokens it can process in one request. If you put a number that is too high here, you will not get the number of prompts you want.
- The number of tokens for gpt-4-0125-preview is 128,000 based on details given here: https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo.
- From a quick internet search, I think this model uses about 1.3 tokens per word. So you might have about 4096/1.3 = 98,461 words to play with if using this model. Remember, the number of tokens or words includes those in the prompt.
- This should be more than enough to generate your entire dataset in one go, so you could try setting examples_per_request to the same number as total_examples. One reason to err on slightly fewer examples is that bigger responses from OpenAI take longer to generate. If you lose your internet connection before generation has ended, you lose your entire generated dataset. If you generate the data in small batches, you will have a partially complete dataset up until the batch that fails.
- This Notebook will make sure that the total number of examples you want to generate as defined in total_examples are generated, but it will make multiple requests to the LLM. Eg if you made total_examples = 100 and examples_per_request = 5, then this code will automatically make 100/5 = 20 requests to the LLM to generate the 100 examples you want.


# Template Creation


Example Template:
> *Please generate a sentence that embodies the concept of {{concept}}, or the opposite of {{concept}} in the context of {{context}}, choosing either with 50% probability. Only write this sentence and nothing else.*

What is a template?
- A generic prompt stucture.
- Any variable/placeholder you might want to pass to your template should be include within a pair of braces like so: {{ your_variable_name }}
- You want to use placeholders where you might want to switch out different parts of the prompt. E.g. you might write {{concept}} instead of politeness or justice or what have you so that you can decide later what to add.
- You will be given the opportunity to fill in placeholders at the end of this form.
- You should write your templates as if you expect only to get one entry of yoiur dataset back. E.g. as if you are asking for one sentence about a concept, not however many you want your dataset to contain. The code will handle creating a dataset for you based on the number you added earlier for dataset size.

In [5]:
ui.create_template_manager()

HTML(value='\n        <style>\n            .widget-label {\n                white-space: normal;\n            …

Rendered Prompt: Please generate a sentence that embodies the concept of rage, or the opposite of rage in the context of professional, choosing either with 50% probability. Only write this sentence and nothing else.


Enter below your prompts for generating dataset labels.

Instructions

- The previous form allows you to generate a dataset of prompts.
- The next form allows you to create questions that will label each prompt from the generated set of prompts.
- Each generated prompt is sent one by one with the question back through the LLM as a new combined prompt to get the label.
- Call a prompt from your generated dataset of prompts P.
- Imagine the generated dataset of prompts are either polite or impolite statements e.g.
- "Would you be so kind as to pass the water please." or "Give me the water now."
- You want to label each prompt as "polite" or "impolite".
- In the following form you will be asked to come up with pairs of "headings" and "labellings"
- A "heading" is the name for the column for the labelling in your dataset.
- A "labelling" is the question you will be asking to label the generated prompt.
- In the example below:


```
prompt, ethical_area, ethical_valence
"Would you be so kind as to pass the water please.", "polite", 1
"Give me the water now.", "impolite", 0
```

- The headings are "ethical_area" and "ethical_valance"
- The labellings that got us the labels might be:
- "Do you think the start of the response in the following text is polite or impolite? Output only the word "polite" for polite, or "impolite" for impolite in single word response within single quote marks. Here is the text: "
- "Do you think the start of the response in the following text is polite or impolite? Output only the number 1 for polite, or 0 for impolite. Here is the text: "
- Call a specific heading H and a specific labelling L.
- P and L would be passed back to the LLM as L + P e.g.
- "Do you think the start of the response in the following text is polite or impolite? Output only the word "polite" for polite, or "impolite" for impolite in single word response within single quote marks. Here is the text: Give me the water now."


Details

- You may need to experiment a little to get good labelling from the LLM.
- It is not certain that the LLM will be able to perform the labelling you are asking for, and if it can it may make mistakes.
- You need to check the labels are correct both in the Review section and after the full Dataset Generation section.
- You need to generate labels for ethical_area and ethical_valency at a minimum for the way the code currently works.
- In the example above, ethical_area always map to the same ethical_valency and vice versa. In other words they are synomyms. They need not be though.
- Your label's name and the prompt to generate it should be enclosed in quote marks. 
- It is a good idea to ask the LLM for the single word or number or whatever else you want so it does not add in unnecessary additional comments such as "Sure, this sentence is impolite" etc which we do not want as part of our label.

In [6]:
ui.display_heading_labelling_form()

HeadingLabellingForm(children=(HTML(value='\n            <h3>Heading-Labelling Form</h3>\n            <p>Use t…

In [7]:
print("H-L Pairs:")
hl_pairs = ui.qa_form.hl_pairs
for k, v in hl_pairs.items():
    print(f"{k}: {v}")

H-L Pairs:
ethical_area: Do you think the start of the response in the following text is rage or calm? Output only the word "rage" for rage, or "calm" for calm. Here is the text:
ethical_valence: Do you think the start of the response in the following text is rage or calm? Output only the number 1 for rage, or 0 for calm. Here is the text:


# Review

You do not need to alter the code in any of the cells from here or on. What you do need to do is as follows: after running the code to the end of this section, just before the Dataset Generation Section, check that the kinds of prompts that are being generated for the new dataset of prompts are as you expected, and that any labelling of the data that is being done looks correct. This is to save you wasting time and money by avoiding generating the whole dataset but then not liking the results.

### !!! CHECK YOUR DATASET SAMPLE HERE BEFORE CONTINUING!!!

If you are happy with the results, continue, if not, go back to the start of the Inputs section and work through to this point again.

In [8]:
dataset_dir = os.path.join("../data/inputs/datasets", nh.filename)
 
# Create the directory if it doesn't exist
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    print(f"Directory created: {dataset_dir}")
else:
    print(f"Directory already exists: {dataset_dir}")

Directory created: ../data/inputs/datasets/testtest


In [9]:
# Generate the dataset by calling the OpenAI API
def generate_dataset_from_prompt(prompt,
                                 generated_dataset_file_path,
                                 client,
                                 model,
                                 temperature,
                                 log_file_path,
                                 i):
    
    # print(client.api_key)

    completion = client.chat.completions.create(
            **{
                "model": model,
                "temperature": temperature,
                "seed": i,
                "messages": [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ]
            }
        )
    
    completion_words = completion.choices[0].message.content.strip()

    # cleaned_completion = completion.choices[0].message.content.strip()[3:-3]
    print(" ")
    print(completion_words)
    print(" ")

    # Open a file in write mode ('w') and save the CSV data
    with open(generated_dataset_file_path+"_"+str(i)+".txt", 'w', newline='', encoding='utf-8') as file:
        file.write(completion_words)

    num_words_in_prompt = count_words_in_string(prompt)
    num_words_in_completion = count_words_in_string(completion_words)
    total_words = num_words_in_prompt + num_words_in_completion

    num_tokens_in_prompt = completion.usage.prompt_tokens
    num_tokens_in_completion = completion.usage.completion_tokens
    total_tokens = num_tokens_in_prompt + num_tokens_in_completion

    prompt_cost = num_tokens_in_prompt*0.01/1000
    completion_cost = num_tokens_in_completion*0.03/1000
    total_cost = prompt_cost + completion_cost
    
    tokens_per_prompt_word = num_words_in_prompt/num_tokens_in_prompt
    tokens_per_completion_word = num_words_in_completion/num_tokens_in_completion

    log = {
            "num_words_in_prompt": num_words_in_prompt,
            "num_words_in_completion": num_words_in_completion,
            "total_words": total_words,
            "num_tokens_in_prompt": num_tokens_in_prompt,
            "num_tokens_in_completion": num_tokens_in_completion,
            "total_tokens": total_tokens,
            "prompt_cost": prompt_cost,
            "completion_cost": completion_cost,
            "total_cost": total_cost,
            "tokens_per_prompt_word": tokens_per_prompt_word,
            "tokens_per_completion_word": tokens_per_completion_word

    }

    for k, v in log.items():
        print(k, v)
    print(" ")

    with open(log_file_path+"_"+str(i)+".txt", 'w') as file:
        file.write(json.dumps(log, indent=4))

def count_words_in_string(input_string):
    words = input_string.split()
    return len(words)

In [10]:
prompt = f"Repeat the following instruction 10 times, always generating a unique answer to the instruction. Begin instruction: {nh.rendered_text} End instruction. Put the result of each instruction within a pair quote marks on a new line as if each was the row of a single column csv and include no other text."
prompt

'Repeat the following instruction 10 times, always generating a unique answer to the instruction. Begin instruction: Please generate a sentence that embodies the concept of rage, or the opposite of rage in the context of professional, choosing either with 50% probability. Only write this sentence and nothing else. End instruction. Put the result of each instruction within a pair quote marks on a new line as if each was the row of a single column csv and include no other text.'

In [12]:
# Generate the sample dataset

# First save the prompt as a text file
with open(dataset_dir+"/prompt.txt", 'w', newline='', encoding='utf-8') as file:
    file.write(prompt)

# Define the file path for the generated dataset
generated_dataset_file_path = os.path.join(dataset_dir, f"{nh.filename}")

# Define the log file path
log_file_path = os.path.join(dataset_dir, "log")

# Define the number of iterations
num_iterations = math.ceil(nh.total_examples/nh.examples_per_request)

start_time = time.time()

# Generate sample dataset
generate_dataset_from_prompt(prompt, generated_dataset_file_path, nh.client, nh.model, nh.temperature, log_file_path, 0)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"The code took {elapsed_time} seconds to run.")

 
"Her calm demeanor in the courtroom was the absolute antithesis of rage."
"His fiery tirade during the meeting was the epitome of professional rage."
"Her peaceful resolution tactics were the perfect counter to workplace rage."
"His reputation for zen-like tranquility was the perfect antonym of professional rage."
"His outburst during the conference call was a startling display of office rage."
"Her tranquility even amidst chaos made her the embodiment of calmness, a stark opposition to rage at work."
"His explosive reaction to the project's failure was a clear indication of his uncontrolable professional rage."
"Her consistent calm and composed demeanor in high-stress situations was a blatant inverse of occupational rage."
"His fierce and confronting argument during the team meeting was a powerful reflection of professional fury."
"Her ability to maintain balance even in high-pressure scenarios made her the office icon of serenity, the exact opposite of professional rage."
 
num_wor

In [13]:
dataset_dir

'../data/inputs/datasets/testtest'

In [14]:
generated_dataset_file_path

'../data/inputs/datasets/testtest/new_template'

# Dataset Generation

In [15]:
prompt = f"Repeat the following instruction {nh.examples_per_request} times, always generating a unique answer to the instruction. Begin instruction: {nh.rendered_text} End instruction. Put the result of each instruction within a pair quote marks on a new line as if each was the row of a single column csv and include no other text."
prompt

'Repeat the following instruction 100 times, always generating a unique answer to the instruction. Begin instruction: Please generate a sentence that embodies the concept of rage, or the opposite of rage in the context of professional, choosing either with 50% probability. Only write this sentence and nothing else. End instruction. Put the result of each instruction within a pair quote marks on a new line as if each was the row of a single column csv and include no other text.'

In [16]:
# Define the number of iterations
num_iterations = math.ceil(nh.total_examples/nh.examples_per_request)
nh.total_examples
start_time = time.time()

# Generate the dataset
for i in range(num_iterations):
    print("Iteration: ", i)
    generate_dataset_from_prompt(prompt, generated_dataset_file_path, nh.client, nh.model, nh.temperature, log_file_path, i)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"The code took {elapsed_time} seconds to run.")

Iteration:  0


KeyboardInterrupt: 

# View Dataset

In [23]:
# Get a list of all the files you want to process
# Makes sure they all have the same prompt context
# eg won;t mix up honest with justice etc
files = [f for f in os.listdir(dataset_dir) 
                      if f.endswith('.txt') and nh.filename in f.lower()]

print(files)

# Define the regular expression pattern
# Get lines that start with a quote,
# then have any number of characters,
# then end with a quote and possibly comma
# We're trying to find all valid CSV lines
pattern = r'^\".*\",?[\r\n]*'

# Open the master CSV file
with open(os.path.join(dataset_dir, nh.filename+"_unlabelled.csv"), "a") as master:
    # Loop over the files
    for file in files:
        # Open the current file and read its contents
        with open(os.path.join(dataset_dir, file), 'r') as f:
            content = f.read()

        # Use the re.findall function to find all matches in the content
        matches = re.findall(pattern, content, re.MULTILINE)        

        # Loop over the matches
        for match in matches:
            
            # Remove any trailing commas and newline characters
            match_cleaned = match.rstrip(',\r\n')
            
            # Append the match to the master CSV file
            master.write(match_cleaned + '\n')

['test_save_5.txt', 'test_save_0.txt', 'test_save_2.txt', 'test_save_1.txt', 'test_save_3.txt', 'test_save_6.txt', 'test_save_4.txt', 'test_save_7.txt', 'test_save_9.txt', 'test_save_8.txt']


## Add optional columns for classification

The columns added here work with the defaults currently hardcoded into the data analysis
This hardcoding will be resolved soon for more flexibility.

I show examples of using am LLM to auto-label and a way to label if you knw in advance which rows are of which kind of label.

### Example of labelling using an LLM

In [24]:
hl_pairs

{'ethical_area': 'Do you think the start of the response in the following text is stupid or clever? Output only the word "stupid" for stupid, or "clever" for clever. Here is the text:',
 'ethical_valence': 'Do you think the start of the response in the following text is stupid or clever? Output only the number 1 for stupid, or 0 for clever. Here is the text:'}

In [25]:
def ask_openai(prompt):
    completion = nh.client.chat.completions.create(
                **{
                    "model": nh.model,
                    "temperature": 0, # For most deterministic results
                    "seed": 0, # For reproducibility
                    "messages": [
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": prompt}
                    ]
                }
            )
    return completion.choices[0].message.content.strip()

input_file_path = generated_dataset_file_path+"_unlabelled.csv"
output_file_path = generated_dataset_file_path+"_labelled.csv"


with open(input_file_path, mode='r', newline='', encoding='utf-8') as infile, \
     open(output_file_path, mode='w', newline='', encoding='utf-8') as outfile:
    
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    header = ["prompt"]

    # Add a header
    for column_name in hl_pairs.keys():
        header.append(column_name)
    
    writer.writerow(header)
    
    for i, row in enumerate(reader):
        print(f"\nRow {i+1}: ")
        for labelling in hl_pairs.values():
            # Assuming each row contains a single column with your text
            statement = row[0]  # Adjust this if your structure is different
            # Here you define the question you want to ask about each row
            labelling_prompt = f"{labelling} {statement}"
            print(labelling_prompt)
            response = ask_openai(labelling_prompt)
            # Add the OpenAI response to the row
            row.append(response)
        writer.writerow(row)


Row 1: 
Do you think the start of the response in the following text is stupid or clever? Output only the word "stupid" for stupid, or "clever" for clever. Here is the text: Surely, at a dinner party, it's wiser to listen and contemplate rather than blabbing about unverified rumors.
Do you think the start of the response in the following text is stupid or clever? Output only the number 1 for stupid, or 0 for clever. Here is the text: Surely, at a dinner party, it's wiser to listen and contemplate rather than blabbing about unverified rumors.

Row 2: 
Do you think the start of the response in the following text is stupid or clever? Output only the word "stupid" for stupid, or "clever" for clever. Here is the text: Frank was aware that in social settings not paying attention to people's personal boundaries was a sign of stupidity.
Do you think the start of the response in the following text is stupid or clever? Output only the number 1 for stupid, or 0 for clever. Here is the text: Fran

# Inspect dataset random sample (set to 10)

In [26]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df = pd.read_csv(generated_dataset_file_path+"_labelled.csv")

num_samples = 10
sample_df = df.sample(num_samples)

sample_df

,prompt,ethical_area,ethical_valence
36,"When everyone's talking at once, she sometimes throws in comments so absurd, it makes you question if she's in the same conversation.",clever,0
33,"In the flood of inane prattles, his words stand out, building bridges between people and their diverging thoughts.",clever,0
14,His habit of interrupting others mid-sentence during a conversation showcased a great degree of social stupidity.,Stupid,1
20,Everyone laughed when Jeff tried to use silverware to eat his soup.,Stupid,1
98,Sending an inappropriate meme in the professional group chat was a classic mistake of social stupidity.,Stupid,1
59,Knowing when not to state the obvious separates the wise from the foolish in a social setting.,clever,0
5,An individual who aims to dominate every conversation reflects stupidity in a social context.,clever,0
66,Forcing unsolicited advice onto others in a social setting aptly illustrates social stupidity.,clever,0
63,"Understanding when to voice your opinion and when to listen in a group discussion, shows the opposite of social stupidity.",clever,0
12,"In the social setting, his reputation for making nonsensical comments reflected ultimate stupidity.",Stupid,1


# Inspect full dataset

In [27]:
df

,prompt,ethical_area,ethical_valence
0,"Surely, at a dinner party, it's wiser to listen and contemplate rather than blabbing about unverified rumors.",clever,0
1,Frank was aware that in social settings not paying attention to people's personal boundaries was a sign of stupidity.,clever,0
2,Sensitivity to different cultures in a social setting is certainly a departure from ignorance.,clever,0
3,"Taking the effort to get a person's name right in a social setting showcases intelligence, a far cry from foolishness.",clever,0
4,"In social settings, talking without considering other's feelings places you in the realm of stupidity.",clever,0
5,An individual who aims to dominate every conversation reflects stupidity in a social context.,clever,0
6,Appreciating diversity in opinions in a social gathering is a quality far beyond stupidity.,clever,0
7,"When in a crowd, knowing when to speak and when to listen is a display of intelligence, not stupidity.",clever,0
8,"Cracking a distasteful joke in a social setting is a clear illustration of thoughtlessness, the cousin of stupidity.",clever,1
9,Understanding social cues and responding appropriately is far from being classified as stupidity.,clever,0


In [ ]:
# Save the dataframe to a CSV file if string issues?